Create a python program that simulates a simple AI agent that can learn to play tic-tac -toe.

The program must Define the game board,
Check if a player has won, Check if the game is a tie, Main game loop, Call the main game loop.

In [1]:
"""
Functions:
- display_board(board)
- available_moves(board)
- make_move(board, idx, player)
- check_winner(board)
- is_tie(board)
- train_q_agent(...)
- play_against_agent(...)
"""

import random
import pickle
import os

# ---------- Helpers for board and rules ----------
WIN_POSITIONS = [
    (0,1,2),(3,4,5),(6,7,8),    # rows
    (0,3,6),(1,4,7),(2,5,8),    # cols
    (0,4,8),(2,4,6)             # diagonals
]

def display_board(board):
    """Pretty-print 3x3 board (board is list of 9 strings)."""
    print()
    print(f" {board[0]} | {board[1]} | {board[2]} ")
    print("---+---+---")
    print(f" {board[3]} | {board[4]} | {board[5]} ")
    print("---+---+---")
    print(f" {board[6]} | {board[7]} | {board[8]} ")
    print()

def board_to_state(board):
    """Convert list board to a string state (immutable key)."""
    return ''.join(board)

def available_moves(board):
    return [i for i, v in enumerate(board) if v == ' ']

def make_move(board, idx, player):
    board[idx] = player

def check_winner(board):
    """Return 'X' or 'O' if there is a winner, otherwise None."""
    for a,b,c in WIN_POSITIONS:
        if board[a] == board[b] == board[c] and board[a] != ' ':
            return board[a]
    return None

def is_tie(board):
    return ' ' not in board and check_winner(board) is None

# ---------- Q-learning agent ----------
def init_q_for_state(Q, state):
    if state not in Q:
        Q[state] = [0.0] * 9  # value for each of 9 actions

def choose_action_epsilon_greedy(Q, state, board, epsilon=0.1):
    moves = available_moves(board)
    init_q_for_state(Q, state)
    if random.random() < epsilon:
        return random.choice(moves)
    # pick best among available moves
    values = [(Q[state][m], m) for m in moves]
    max_val = max(values)[0]
    best_moves = [m for v,m in values if v == max_val]
    return random.choice(best_moves)

def choose_action_greedy(Q, state, board):
    moves = available_moves(board)
    init_q_for_state(Q, state)
    values = [(Q[state][m], m) for m in moves]
    max_val = max(values)[0]
    best_moves = [m for v,m in values if v == max_val]
    return random.choice(best_moves)

def train_q_agent(episodes=5000, alpha=0.3, gamma=0.9, epsilon=0.2, eps_decay=0.9997, verbose=False):
    """
    Train a Q-table by playing the Q-agent vs a random agent.
    Returns Q (dict).
    """
    Q = {}
    for ep in range(episodes):
        board = [' '] * 9
        # Randomly choose who starts to avoid bias
        q_player = 'X' if random.random() < 0.5 else 'O'
        other = 'O' if q_player == 'X' else 'X'
        state = board_to_state(board)
        q_history = []  # only store Q-agent moves as (state, action)
        current = 'X'
        while True:
            if current == q_player:
                action = choose_action_epsilon_greedy(Q, state, board, epsilon)
                make_move(board, action, q_player)
                q_history.append((state, action))
            else:
                # random opponent
                moves = available_moves(board)
                action = random.choice(moves)
                make_move(board, action, other)

            winner = check_winner(board)
            if winner or is_tie(board):
                # set reward from Q-agent perspective
                if winner == q_player:
                    reward = 1.0
                elif winner is None:
                    reward = 0.5  # tie
                else:
                    reward = -1.0

                # Back-propagate reward through agent's history (simple episodic update)
                # We go backwards, decaying the reward by gamma
                r = reward
                for (s,a) in reversed(q_history):
                    init_q_for_state(Q, s)
                    Q[s][a] += alpha * (r - Q[s][a])
                    r *= gamma
                break

            # continue to next turn
            state = board_to_state(board)
            current = other if current == q_player else q_player

        # decay epsilon slowly (less exploration over time)
        epsilon *= eps_decay
        if verbose and (ep+1) % (episodes//5) == 0:
            print(f"Episode {ep+1}/{episodes}, epsilon={epsilon:.4f}")

    return Q

# ---------- Play against trained agent ----------
def play_against_agent(Q, human_player='X'):
    """
    Let a human play against the trained Q-agent.
    human_player: 'X' or 'O'
    """
    agent_player = 'O' if human_player == 'X' else 'X'
    board = [' '] * 9
    current = 'X'
    print("Positions are numbered 0..8 like this:")
    print(" 0 | 1 | 2 ")
    print("---+---+---")
    print(" 3 | 4 | 5 ")
    print("---+---+---")
    print(" 6 | 7 | 8 ")
    print()
    while True:
        display_board(board)
        if current == human_player:
            moves = available_moves(board)
            move = None
            while move not in moves:
                try:
                    move = int(input(f"Your turn ({human_player}). Choose move (0-8): ").strip())
                except:
                    move = None
            make_move(board, move, human_player)
        else:
            s = board_to_state(board)
            if s not in Q:
                # fallback to random if agent hasn't seen this state
                move = random.choice(available_moves(board))
            else:
                move = choose_action_greedy(Q, s, board)
            print(f"Agent ({agent_player}) chooses {move}")
            make_move(board, move, agent_player)

        winner = check_winner(board)
        if winner or is_tie(board):
            display_board(board)
            if winner:
                print(f"Winner: {winner}")
            else:
                print("It's a tie!")
            break
        current = 'O' if current == 'X' else 'X'

# ---------- Save/Load Q-table helpers ----------
def save_q_table(Q, filename="q_table.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(Q, f)

def load_q_table(filename="q_table.pkl"):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)
    return None

# ---------- Main ----------
if __name__ == "__main__":
    print("Tic-Tac-Toe with a simple learning agent (Q-Learning).")
    # 1) Train
    print("Training agent... (this can take a few seconds depending on episodes)")
    Q = train_q_agent(episodes=7000, alpha=0.3, gamma=0.9, epsilon=0.3, verbose=True)
    save_q_table(Q, "q_table.pkl")
    print("Training complete and Q-table saved as q_table.pkl")

    # 2) Play against trained agent
    while True:
        ans = input("Play a game against the agent? (y/n): ").strip().lower()
        if ans in ("y","yes"):
            # ask which side
            side = input("Do you want to be X or O? (X plays first): ").strip().upper()
            if side not in ("X","O"):
                side = "X"
            play_against_agent(Q, human_player=side)
        else:
            print("Goodbye!")
            break


Tic-Tac-Toe with a simple learning agent (Q-Learning).
Training agent... (this can take a few seconds depending on episodes)
Episode 1400/7000, epsilon=0.1971
Episode 2800/7000, epsilon=0.1295
Episode 4200/7000, epsilon=0.0851
Episode 5600/7000, epsilon=0.0559
Episode 7000/7000, epsilon=0.0367
Training complete and Q-table saved as q_table.pkl
Play a game against the agent? (y/n): 
Goodbye!
